<a href="https://colab.research.google.com/github/pb-roshith/Building-Machine-Learning-Pipeline/blob/main/Building_Machine_Learning_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [3]:
df.shape

(10000, 14)

In [4]:
df.isna().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
x = df.drop(columns=['Exited'])
y = df['Exited']

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [11]:
x_train.shape[0], x_test.shape[0]

(8000, 2000)

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [13]:
num_pipeline = Pipeline([
    ('num_imputation', SimpleImputer(strategy='mean')),
    ('feature_scaling', MinMaxScaler()),
    ('pca', PCA(0.98))
])

In [14]:
num_pipeline

Pipeline(steps=[('num_imputation', SimpleImputer()),
                ('feature_scaling', MinMaxScaler()),
                ('pca', PCA(n_components=0.98))])

In [15]:
cat_pipeline = Pipeline([
    ('cat_imputation', SimpleImputer(fill_value='missing', strategy='constant')),
    ('one_hot_encoding', OneHotEncoder(handle_unknown='ignore'))
])

In [16]:
cat_pipeline

Pipeline(steps=[('cat_imputation',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('one_hot_encoding', OneHotEncoder(handle_unknown='ignore'))])

In [17]:
num_cols = x.select_dtypes(include=np.number).columns.tolist()
cat_cols = x.select_dtypes(include='object').columns.tolist()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [19]:
preprocessor = ColumnTransformer([
    ('categorical', cat_pipeline, cat_cols),
    ('numerical', num_pipeline, num_cols)
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('estimator', RandomForestClassifier())
])

In [20]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('cat_imputation',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoding',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Geography', 'Gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('num_imputation',
                                                                   SimpleImputer()),
                                                                  ('feature_scaling',
                                                                   MinMaxScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.98))]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('estimator', RandomForestClassifier())])

In [21]:
pipe.predict(x_test)

array([0, 0, 0, ..., 0, 0, 1])

In [22]:
pipe.score(x_test, y_test) * 100

86.1

In [27]:
parameters = {
    'estimator__n_estimators': [100, 150],
    'estimator__max_depth': [7, 10],
    'estimator__min_samples_split': [2, 3],
    'estimator__max_features': [4, 6, 8]
}

grid_search = GridSearchCV(
    pipe,
    param_grid = parameters,
    cv=3,
    n_jobs = -1
)

In [28]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('cat_imputation',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('one_hot_encoding',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Geography',
                                                                          'Gender']),
                                                                        ('numerical',
                                                                         Pipeline(steps=[('num_imputation',
                                                                                          SimpleImputer()),
                                                                                         ('...
                                                                                          MinMaxScaler()),
                                                                                         ('pca',
                                                                                          PCA(n_components=0.98))]),
                                                                         ['CreditScore',
                                                                          'Age',
                                                                          'Tenure',
                                                                          'Balance',
                                                                          'NumOfProducts',
                                                                          'HasCrCard',
                                                                          'IsActiveMember',
                                                                          'EstimatedSalary'])])),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [7, 10],
                         'estimator__max_features': [4, 6, 8],
                         'estimator__min_samples_split': [2, 3],
                         'estimator__n_estimators': [100, 150]})

In [29]:
grid_search.best_params_

{'estimator__max_depth': 10,
 'estimator__max_features': 4,
 'estimator__min_samples_split': 3,
 'estimator__n_estimators': 150}

In [30]:
pipe2 = Pipeline([
    ('preprocessor', preprocessor),
    ('estimator', RandomForestClassifier(n_estimators=150,
                                         max_features=6,
                                         min_samples_split=3,
                                         max_depth=4))
])

In [31]:
pipe2.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('cat_imputation',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoding',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Geography', 'Gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('num_imputation',
                                                                   SimpleImputer()),
                                                                  ('feature_scaling',
                                                                   MinMaxScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.98))]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('estimator',
                 RandomForestClassifier(max_depth=4, max_features=6,
                                        min_samples_split=3,
                                        n_estimators=150))])

In [32]:
pipe2.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
pipe2.score(x_test, y_test) * 100

85.25